In [1]:
import pandas as pd
from pathlib import Path
import numpy as np 
%matplotlib inline

In [2]:
# Set file paths
hd_data = Path("./Resources/HD.csv")
jnj_data = Path("./Resources/JNJ.csv")
intc_data = Path("./Resources/INTC.csv")
amd_data = Path("./Resources/AMD.csv")
mu_data = Path("./Resources/MU.csv")
nvda_data = Path("./Resources/NVDA.csv")
tsm_data = Path("./Resources/TSM.csv")


# Read the individual CSV datasets
hd = pd.read_csv(hd_data, index_col="date", infer_datetime_format=True, parse_dates=True)
jnj = pd.read_csv(jnj_data, index_col="date",infer_datetime_format=True, parse_dates=True)
intc = pd.read_csv(intc_data, index_col="date",infer_datetime_format=True, parse_dates=True)
amd = pd.read_csv(amd_data, index_col="date",infer_datetime_format=True, parse_dates=True)
mu = pd.read_csv(mu_data, index_col="date",infer_datetime_format=True, parse_dates=True)
nvda = pd.read_csv(nvda_data, index_col="date",infer_datetime_format=True, parse_dates=True)
tsm = pd.read_csv(tsm_data, index_col="date",infer_datetime_format=True, parse_dates=True)

In [3]:
combined_df = pd.concat([jnj, hd, intc, amd], axis="columns", join="inner")
combined_df.sort_index(inplace=True)
combined_df.head()

combined_df2 = pd.concat([mu,nvda,tsm], axis="columns", join="inner")
combined_df2.sort_index(inplace=True)
combined_df2.head()

# Creates a dictionary of dataframes 

dataframe_dictionary = {"First_df": combined_df,
                        "Second_df" : combined_df2}


 

In [4]:
## Function that calculates log returns  
def calculateLogreturns(df):
    
    returns1 = df/df.shift(1)
    log_Returns = np.log(returns1)
    return log_Returns

    

In [5]:
## Function that runs Efficient Frontier Simulation and returns weighted index, return, expected return  

def efficient_frontier(calculateLogreturns, number_columns):
        number_portfolios = 10
        weight = np.zeros((number_portfolios, number_columns))

        expectedReturn = np.zeros(number_portfolios)
        expectedVolatility = np.zeros(number_portfolios)
        sharpeRatio = np.zeros(number_portfolios)

    # meanLogRet = log_Returns.mean()
        meanLogRet = calculateLogreturns(df).mean()
        Sigma = calculateLogreturns(df).cov()

        for k in range(number_portfolios):
            # Generate random weight vector
            w = np.array(np.random.random(number_columns))
            w = w /np.sum(w)
            weight[k,:] = w
            # Expected log return
            expectedReturn[k] = np.sum(meanLogRet * w)
    
            # Expected volatility
            expectedVolatility[k] = np.sqrt(np.dot(w.T, np.dot(Sigma, w)))
   
            # Sharpe Ratio
            sharpeRatio[k] = expectedReturn[k]/expectedVolatility[k]
    
        WeightIndex = sharpeRatio.argmax()
        Final_Index = weight[WeightIndex,:]
        return Final_Index, expectedReturn, expectedVolatility

   

In [9]:
for name_of_df, df in dataframe_dictionary.items():
    
    print(df)
    
    columns = df.columns
    shape = df.shape
    number_columns = shape[1]
    calculateLogreturns(df)
    
    
    
    Weights, Returns, Volatility = efficient_frontier(calculateLogreturns, number_columns)
   
print(number_columns)

    outputdataframe = pd.DataFrame(a)
    outputdataframe = outputdataframe.T
    outputdataframe.columns = columns
    outputdataframe = outputdataframe.reset_index(drop = True)
    
   
   
    
    outputdataframe.to_csv(f"{name_of_df}.csv")

#     # Create .csv file
#     file_path = Path(f"./EfficientFrontier_Output/{i}.csv")
#     outputdataframe.to_csv(file_path)

                JNJ      HD   INTC    AMD
date                                     
2009-05-14  11.9665   24.53  15.54   4.24
2009-05-15  11.7316   24.40  15.19   4.01
2009-05-18  12.1925   26.02  15.52   4.14
2009-05-19  12.3875   24.63  15.65   4.39
2009-05-20  12.3299   23.83  15.58   4.50
...             ...     ...    ...    ...
2019-05-08  66.0200  195.17  49.24  27.09
2019-05-09  66.0400  194.58  46.62  27.21
2019-05-10  67.2200  194.58  46.20  27.96
2019-05-13  66.4900  190.34  44.76  26.24
2019-05-14  66.9700  191.62  45.17  27.32

[2517 rows x 4 columns]
[0.28045561 0.34576939 0.19240853 0.18136647]
[0.00065749 0.00069036 0.0006726  0.00062042 0.00058528 0.00065106
 0.00074199 0.0006373  0.0007421  0.0006677 ]
[0.01593676 0.01368362 0.01723293 0.01534288 0.01435245 0.01310419
 0.01530002 0.01766109 0.01487906 0.01859855]
4
               MU    NVDA      TSM
date                              
2009-05-14   4.55    8.52  10.2488
2009-05-15   4.24    8.74   9.9701
2009-05-18   4.